## Function Definition

In [ ]:
def getProbTable(x):
    val_counts = (pd.DataFrame(x.value_counts())).reset_index()
    val_counts.columns = ['Value','Count']
    val_counts["Prob"] = val_counts['Count'].map(lambda x: x/float(sum(val_counts.Count)))
    val_counts.drop("Count",axis=1,inplace=True)
    return val_counts.set_index('Value').T.to_dict('list')

## Load Data

In [ ]:
import pandas as pd
import numpy as np
import csv

In [ ]:
print('Load data...')
train = pd.read_csv("./input/train.csv")
target = train['target'].values
id_train= train['ID'].values
train = train.drop(['ID','target'],axis=1)
test = pd.read_csv("./input/test.csv")
id_test = test['ID'].values
test = test.drop(['ID'],axis=1)

## Filling missing values in categorical parameters
## Categorical to numerical conversion

In [ ]:
obj_params = list()

for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    if train_series.dtype == 'O':
        obj_params.append(train_name)

#train_obj = train[obj_params]
#test_obj = test[obj_params]

train_obj = train[obj_params]
test_obj = test[obj_params]

#train_obj = train_obj.drop("v22",axis=1)
#test_obj = test_obj.drop("v22",axis=1)


In [ ]:
#check number of NAs in each column
for (train_obj_name, train_obj_series), (test_obj_name, test_obj_series) in zip(train_obj.iteritems(),test_obj.iteritems()):
    
    if(train_obj_series.isnull().sum()>(len(train_obj_series)/3)):
        train_obj.loc[train_obj_series.isnull(), train_obj_name] = "NOT_CAPTURED"
    else:
        #fill others with most frequent value
        train_obj.loc[train_obj_series.isnull(), train_obj_name] = train_obj_series.mode().iloc[0]
    
    if(test_obj_series.isnull().sum()>(len(test_obj_series)/3)):
        test_obj.loc[test_obj_series.isnull(), test_obj_name] = "NOT_CAPTURED"
    else:
        #fill others with most frequent value
        test_obj.loc[test_obj_series.isnull(), test_obj_name] = test_obj_series.mode().iloc[0]

for (train_obj_name, train_obj_series), (test_obj_name, test_obj_series) in zip(train_obj.iteritems(),test_obj.iteritems()):
    repl_nums = getProbTable(train_obj_series)
    train_obj[train_obj_name].replace(repl_nums,inplace=True)

    repl_nums = getProbTable(test_obj_series)
    test_obj[test_obj_name].replace(repl_nums,inplace=True)

In [28]:
train = train.drop(obj_params,axis=1)
test = test.drop(obj_params,axis=1)

## Filling missing values in numerical columns

In [15]:
#Add number of missing values per row as a parameter
train['na_count'] = train.isnull().sum(axis=1)
test['na_count'] = test.isnull().sum(axis=1)

In [18]:
for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
    tmp_len = len(train[train_series.isnull()])
    if tmp_len>0:
            train.loc[train_series.isnull(), train_name] = -9999 #train_series.mean()
    tmp_len = len(test[test_series.isnull()])
    if tmp_len>0:
            test.loc[test_series.isnull(), test_name] = -9999 #train_series.mean()  #TODO

In [29]:
train = pd.concat([train,train_obj],axis=1)
test = pd.concat([test,test_obj],axis=1)
train['target'] = target
#test['target'] = id_test
train['ID'] = id_train
test['ID'] = id_test

train.to_csv("train_CatgProbs.csv",index=False)
test.to_csv("test_CatgProbs.csv",index=False)

In [31]:
train['target'].value_counts()

1    87021
0    27300
Name: target, dtype: int64